In [148]:
import numpy as np   
import yfinance as yf
import pandas as pd
from pandas_datareader import data as pdr
yf.pdr_override() # this is just for  two libraries are commonly used together. it helps them work better


In [149]:
df1 = pdr.get_data_yahoo('AAPL','2011-01-03','2021-10-31')
df1 = df1[['Open','Close']] # get the colom open close
df1 =df1.rename(columns ={'Open':'Open_AAPL','Close':'Close_AAPL'})
df2 = pdr.get_data_yahoo('MSFT','2011-01-03','2021-10-31')
df2 = df2[['Open','Close']]
df2 =df2.rename(columns ={'Open':'Open_MSFT','Close':'Close_MSFT'})
df =df1.merge(df2, on = 'Date')       # on = دمج الجدولين والرابط ال date 
df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open_AAPL,Close_AAPL,Open_MSFT,Close_MSFT
Date,,,,
2011-01-03,11.630000,11.770357,28.049999,27.980000
2011-01-04,11.872857,11.831786,27.940001,28.090000
2011-01-05,11.769643,11.928571,27.900000,28.000000
2011-01-06,11.954286,11.918929,28.040001,28.820000
2011-01-07,11.928214,12.004286,28.639999,28.600000
...,...,...,...,...
2021-10-25,148.679993,148.639999,309.359985,308.130005
2021-10-26,149.330002,149.320007,311.000000,310.109985
2021-10-27,149.360001,148.850006,316.000000,323.170013


In [150]:
two_years =  df.loc['2011-01-01':'2012-12-31'] # choes only two years 
two_years = two_years[['Close_AAPL','Close_MSFT']]                                      # for the return we use the close returns_appl = appl.pct_change()
two_years ['Return_AAPL'] = two_years['Close_AAPL'].pct_change()
two_years ['Return_MSFT'] = two_years['Close_MSFT'].pct_change() # we need w4 a1 
two_years = two_years[['Return_AAPL','Return_MSFT']]
two_years = two_years.dropna()
two_years



,Return_AAPL,Return_MSFT
Date,,
2011-01-04,0.005219,0.003931
2011-01-05,0.008180,-0.003204
2011-01-06,-0.000808,0.029286
2011-01-07,0.007161,-0.007634
2011-01-10,0.018833,-0.013287
...,...,...
2012-12-24,0.001617,-0.014208
2012-12-26,-0.013784,-0.007391
2012-12-27,0.004016,0.003723


In [151]:
matrix_of_return_values = np.asmatrix(two_years)
matrix_of_return_values = matrix_of_return_values.T

no_of_asset_classes = 2
no_of_portfolios = 10000
collection_of_random_asset_weights = []
portfolio_means = []
portfolio_standard_deviations = []
portfolio_sharpe_ratios = []

In [152]:

def get_random_asset_weights(no_of_asset_classes):
    random_asset_weights = np.random.rand(no_of_asset_classes)
    random_asset_weights /= sum(random_asset_weights)  
    random_asset_weights = np.asmatrix(random_asset_weights)  
    return random_asset_weights    


def get_portfolio_statistics(asset_weights, matrix_of_return_values): 
    asset_means = np.asmatrix(np.mean(matrix_of_return_values, axis = 1))
    asset_covariances = np.asmatrix(np.cov(matrix_of_return_values))
    portfolio_mean = asset_weights * asset_means
    portfolio_mean = portfolio_mean.item()
    portfolio_standard_deviation =np.sqrt(asset_weights * asset_covariances * asset_weights.T) 
    portfolio_standard_deviation = portfolio_standard_deviation.item()
    portfolio_sharpe_ratio = portfolio_mean /portfolio_standard_deviation 
    return portfolio_mean, portfolio_standard_deviation, portfolio_sharpe_ratio


while len(collection_of_random_asset_weights) < no_of_portfolios:
    random_asset_weights = get_random_asset_weights(no_of_asset_classes)
    collection_of_random_asset_weights.append(random_asset_weights) 
    A,B,C=get_portfolio_statistics(random_asset_weights,matrix_of_return_values)
    portfolio_means.append(A)
    portfolio_standard_deviations.append(B)
    portfolio_sharpe_ratios.append(C)


   

In [153]:
index = portfolio_sharpe_ratios.index(max(portfolio_sharpe_ratios))
print(collection_of_random_asset_weights[index])  # 10 000 port we put 99 of our monay in apple

[[9.99883417e-01 1.16583225e-04]]


In [154]:
some_weights =pd.DataFrame({'AAPL':[0.75],'MSFT':[0.25]},index=[two_years.last_valid_index()])
some_weights

,AAPL,MSFT
2012-12-31,0.75,0.25


In [155]:
capital = 10000
allocations = capital*some_weights
allocations
no_of_shares= allocations  


In [156]:
#print(df['Close_AAPL'].loc[some_weights.last_valid_index()])
#print(df['Close_MSFT'].loc[some_weights.last_valid_index()])
#no_of_shares['AAPL'].iloc[0] /= df['Close_AAPL'].loc[some_weights.last_valid_index()]
#no_of_shares['MSFT'].iloc[0] /= df['Close_MSFT'].loc[some_weights.last_valid_index()]
#no_of_shares


In [157]:
#another way to do it with loop (best way)
no_of_shares= allocations / df[[f'Close_{sym}'for sym in some_weights.columns]].loc[some_weights.last_valid_index()].values

no_of_shares

,AAPL,MSFT
2012-12-31,394.610752,93.597907


In [159]:
eight_years = df.loc['2012-12-31':]
eight_years['Open']= eight_years['Open_AAPL']*no_of_shares['AAPL'].iloc[0]+eight_years['Open_MSFT']*no_of_shares['MSFT'].iloc[0]
eight_years['Close']= eight_years['Close_AAPL']*no_of_shares['AAPL'].iloc[0]+eight_years['Close_MSFT']*no_of_shares['MSFT'].iloc[0]
eight_years = eight_years[['Open','Close']]

eight_years

<ipython-input-159-5fdf78183ef2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eight_years['Open']= eight_years['Open_AAPL']*no_of_shares['AAPL'].iloc[0]+eight_years['Open_MSFT']*no_of_shares['MSFT'].iloc[0]
<ipython-input-159-5fdf78183ef2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eight_years['Close']= eight_years['Close_AAPL']*no_of_shares['AAPL'].iloc[0]+eight_years['Close_MSFT']*no_of_shares['MSFT'].iloc[0]


,Open,Close
Date,,
2012-12-31,9683.790407,10000.000000
2013-01-02,10355.661647,10322.786098
2013-01-03,10307.514887,10190.488844
2013-01-04,10120.062557,9929.945971
2013-01-07,9862.287659,9881.577228
...,...,...
2021-10-25,87626.170692,87495.265314
2021-10-26,88036.173232,87948.925784
2021-10-27,88516.000605,88985.849537


In [162]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression


In [161]:
short_time = 5
long_time = 15
eight_years['Close_short_Rolling']=eight_years['Close'].rolling(window=short_time).mean()
eight_years['Close_Long_Rolling']=eight_years['Close'].rolling(window=long_time).mean()
eight_years = eight_years.dropna()
eight_years

,Open,Close,Close_short_Rolling,Close_Long_Rolling
Date,,,,
2013-01-22,9666.108705,9655.028515,9589.089545,9815.460021
2013-01-23,9716.645010,9828.305163,9675.753203,9804.013698
2013-01-24,9075.553113,8935.115641,9530.109208,9711.502335
2013-01-25,8947.206594,8808.844482,9364.891508,9619.392710
2013-01-28,8792.121445,8951.880244,9235.834809,9554.188329
...,...,...,...,...
2021-10-25,87626.170692,87495.265314,87680.815288,85365.721147
2021-10-26,88036.173232,87948.925784,87760.205037,85714.925810
2021-10-27,88516.000605,88985.849537,88022.868655,86082.701927


In [165]:
X = eight_years.copy()
y = np.where(X.Close.shift(-1)>=X.Close,1,-1)
index = int(0.70*len(X))
X_train = X.iloc[:index]
X_test= X.iloc[index:]
y_train = y[:index]
y_test= y[index:]
logistic =LogisticRegression()
logistic = logistic.fit(X_train,y_train)
predictions = logistic.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))
print(logistic.score(X_test,y_test))



[[ 48 247]
 [ 46 323]]
0.5587349397590361
